In [1]:
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess 
from gensim.models.coherencemodel import CoherenceModel 
from gensim.models.ldamodel import LdaModel
import spacy
import nltk
import pyLDAvis 
import warnings
import pickle
import json
from gensim.models import TfidfModel
import matplotlib.pyplot as plt
import re
from pprint import pprint
import pandas as pd

warnings.filterwarnings("ignore")

In [79]:
!pip install xone

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached tqdm-4.63.1-py2.py3-none-any.whl (76 kB)
     |████████████████████████████████| 7.3 MB 1.4 MB/s eta 0:00:01
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
     |████████████████████████████████| 3.0 MB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 899 kB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 7.1 MB/s  eta 0:00:01
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24580 sha256=f23b0f9c3d7949db42d3c6ac140c760c3803ffd56bd34f7215ceff79ff426f97
  Stored in directory: /Users/sam/Library/Caches/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built parse
ERROR: responses 0.18.0 has requirement urllib3>=1.25.10, but you'll have urllib3 1.24.2 which is incompatible.
  Attempting uninst

In [2]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    # import the English spacy object as en
    en = spacy.load('en_core_web_sm', disable=["parser", "ner"])
    # save spacy's stopwords to stopwords
    stopwords = en.Defaults.stop_words # spacy has a longer list of stopwords than NLTK
    # append more stopwords to our stopwords variable
    #stopwords |= {"other", "when", "so", "such", "therefore", "electronic", "ebook", 
    #              "website", "licence", "comply", "format", "table", "copy"}
    texts_out = []
    i = 1
    l = len(texts)
    for text in texts:
        doc = en(text) # use the spacy to create an object that contains essential metadata about our corpus
        new_text = [] 
        # lemmatize
        for token in doc: # iterate over each of the words in the doc object
            if token.pos_ in allowed_postags: # if the token.part.of.speech is in our allowed postags
                new_text.append(token.lemma_) # then append the lemmatized form to our new text.
        # remove stopwords (the spacy way)
        cleaned_text = []
        for text in new_text:
            lexeme = en.vocab[text] # lexeme is a unit of language
            if lexeme.is_stop == False:
                cleaned_text.append(text)
        final = " ".join(cleaned_text) # finally, we reconstute the lemmas as a text again
        texts_out.append(final)
        if i % 100 == 0:
            print(round(i/l, 4)*100, '%')
        i += 1
    return texts_out # this returns your lemmatized text with the stopwords removed


def gen_words(texts): # pass in your texts as an argument
    final = []
    for text in texts: # iterate over the texts
        new = gensim.utils.simple_preprocess(text, deacc=True) 
        final.append(new)
    return final # turns your texts into preprocessed, individual words

def make_bigrams(texts):
    # identify and reconsitute all of your bigrams
    return [bigram[doc] for doc in texts] #important to use square brackets for list comprehension

def make_trigrams(texts):
    return[trigram[bigram[doc]] for doc in texts]


In [175]:
dfs = []

for fn in ['WSJ_Art_Rep_Strings_Even_First15_2006_2016.csv', 'WSJ_Art_Rep_Strings_Evens_Last25_2006_2016.csv', \
          'WSJ_Art_Strings_Rep_Odds_First15_2007_2017.csv', 'WSJ_Art_Strings_Rep_Odds_Last25_2007_2017.csv']:
    dfs.append(pd.read_csv(fn, encoding = "ISO-8859-1"))
    
df = pd.concat(dfs, ignore_index=True)


In [176]:
print(len(df))
df = df.dropna(subset = ['text'])
df = df.drop_duplicates()
print(len(df))

136601
113429


In [178]:
rep_df = df.reset_index().filter(['link', 'text'])
rep_df

,link,text
0,/docview/398950000/A48EE535221A46EEPQ/2?accoun...,New online tools can help investors make sens...
1,/docview/399055896/A48EE535221A46EEPQ/3?accoun...,Happy New Year: Tax relief arrives today for ...
2,/docview/399024538/A48EE535221A46EEPQ/4?accoun...,When Lucille Lieberman's husband suffered a s...
3,/docview/399004443/A48EE535221A46EEPQ/5?accoun...,Every new year presents a new opportunity to ...
4,/docview/398962113/A48EE535221A46EEPQ/6?accoun...,"It's January 1, which means financial comment..."
...,...,...
113424,/docview/1982157166/5E9CBB03258C41D3PQ/83?acco...,"William H. McRaven, 62, is a retired four-sta..."
113425,/docview/1982159541/5E9CBB03258C41D3PQ/84?acco...,[A look back at the news from this year's Wal...
113426,/docview/1982157194/5E9CBB03258C41D3PQ/85?acco...,[Provided by the National Museum of Mathemati...
113427,/docview/1982157175/5E9CBB03258C41D3PQ/86?acco...,History can't quite make up its mind about th...


In [179]:
# links0 = pd.read_csv('WSJ_Art_Links.csv')
# links.tail()
# links1 = pd.read_csv('WSJ_Art_Links_1.csv')
# links1.tail()
#links_df = pd.merge(links0, links1)
# links_df.tail()
dfs = []
for year in range(2007,2018):
    if year == 2008:
        continue
    fn = "WSJ_Art_Links_" + str(year) +".csv"
    dfs.append(pd.read_csv(fn, encoding = "ISO-8859-1"))

dfs.append(pd.read_csv('WSJ_Art_Links_2014_part_2.csv', encoding = "ISO-8859-1"))
dfs.append(pd.read_csv('WSJ_Art_Links.csv', encoding="ISO-8859-1"))
print(dfs)
links = pd.concat(dfs, ignore_index=True)

[       year  month  day                                               link  \
0      2007      1    1  /docview/399037855/27F5A241C5F14C8DPQ/1?accoun...   
1      2007      1    2  /docview/399051820/32E57EEEBF714480PQ/1?accoun...   
2      2007      1    2  /docview/399102696/32E57EEEBF714480PQ/2?accoun...   
3      2007      1    2  /docview/399096388/32E57EEEBF714480PQ/3?accoun...   
4      2007      1    2  /docview/399037309/32E57EEEBF714480PQ/4?accoun...   
...     ...    ...  ...                                                ...   
29856  2007     12   31  /docview/399019134/272E1A7DE5A64FC0PQ/75?accou...   
29857  2007     12   31  /docview/399062132/272E1A7DE5A64FC0PQ/76?accou...   
29858  2007     12   31  /docview/399062409/272E1A7DE5A64FC0PQ/77?accou...   
29859  2007     12   31  /docview/399009311/272E1A7DE5A64FC0PQ/78?accou...   
29860  2007     12   31  /docview/399100232/272E1A7DE5A64FC0PQ/79?accou...   

                                                   title  \
0 

In [180]:
colsToKeep = ["year", "month", "day", "link", "title", "author"]
links = links[colsToKeep]
links.tail()
links['year'].value_counts(normalize=True) * 100

2010.0    10.715349
2012.0     9.580396
2013.0     9.437053
2011.0     9.427745
2008.0     9.414094
2009.0     9.327840
2007.0     9.264856
2016.0     9.120272
2006.0     8.940007
2017.0     8.596853
2014.0     6.175536
Name: year, dtype: float64

In [181]:
merged = rep_df.merge(links, how='inner', on='link')


In [182]:
merged['datetime'] = pd.to_datetime(merged[['year', 'month', 'day']])

In [183]:
merged = merged.sort_values('datetime')
merged

,link,text,year,month,day,title,author,datetime
0,/docview/398950000/A48EE535221A46EEPQ/2?accoun...,New online tools can help investors make sens...,2006.0,1.0,1.0,Tip of the Week: Fund Fees Get Clearer,"Laise, Eleanor.",2006-01-01
8,/docview/399055958/A48EE535221A46EEPQ/10?accou...,The number of telemarketers calling Craig Law...,2006.0,1.0,1.0,MarketWatch: Hello . . . It's the Do-Not-Call ...,"Coombes, Andrea.",2006-01-01
7,/docview/399056023/A48EE535221A46EEPQ/9?accoun...,"The yield curve inverted last week, meaning t...",2006.0,1.0,1.0,In Translation: The Yield Curve Has Inverted; ...,"Ip, Greg.",2006-01-01
6,/docview/399056056/A48EE535221A46EEPQ/8?accoun...,Recalls and safety alerts have roiled the mar...,2006.0,1.0,1.0,Barron's Insight: Medtronic's Looking Good,"Alpert, Bill.",2006-01-01
5,/docview/399055991/A48EE535221A46EEPQ/7?accoun...,Putting your investing on autopilot is the ea...,2006.0,1.0,1.0,SmartMoney: Invest Without Willpower,"Todorova, Aleksandra.",2006-01-01
...,...,...,...,...,...,...,...,...
76160,/docview/1982158528/5E9CBB03258C41D3PQ/15?acco...,"WEST PALM BEACH, Fla. -- Vice President Mike ...",2017.0,12.0,30.0,U.S. News: Vice President Amasses War Chest,"Nicholas, Peter.",2017-12-30
102600,/docview/1982157624/5E9CBB03258C41D3PQ/51?acco...,"The U.S. dollar fell Friday, posting its larg...",2017.0,12.0,30.0,Currencies: Dollar Posts Its Biggest Annual De...,"Kruger, Daniel.",2017-12-30
102601,/docview/1982160021/5E9CBB03258C41D3PQ/52?acco...,This year has been kind to investors. Stocks ...,2017.0,12.0,30.0,"Despite Rally, Some Losers","Eisen, Ben.",2017-12-30
102603,/docview/1982157271/5E9CBB03258C41D3PQ/54?acco...,Companies returned to the U.S. market for ini...,2017.0,12.0,30.0,IPOs Stage a Recovery,"Dulaney, Chelsey.",2017-12-30


In [96]:
merged['year'].value_counts(normalize=True) * 100

2013.0    11.123021
2017.0    11.070402
2011.0    10.936906
2012.0    10.895006
2006.0    10.602680
2016.0    10.067722
2010.0     9.903045
2007.0     9.340804
2008.0     9.026066
2014.0     7.034348
Name: year, dtype: float64

We see that there are roughly the same number of articles for each year except for 2014. 

Now we will start cleaning, using the same steps as the original paper (denoted in Appendix A)

Step 1, remove all articles prior to Jan 1984 and June 2017 

We are leaving June 2017 for now since our dataset is significantly shorter

\2. Replace all non-alphabetical characters with an empty string and set the remaining characters to lower-case


In [187]:
import re
merged['alpha'] = merged['text'].str.replace(r'[^A-Za-z ]+', '', regex=True)
merged['alpha'] = merged['alpha'].str.lower()                              

In [188]:
merged['alpha'].iloc[0]

' new online tools can help investors make sense of mutualfund fees the national association of securities dealers which regulates the brokerage industry has launched two free tools at wwwnasdcom a mutualfund expense analyzer estimates the total dollar cost of holding specific funds  up to three at a time  based on a purchase amount and holding period you select the nasds breakpoint search tool shows how much you must invest in a particular fund to receive any discounts on sales charges the amounts at which those discounts kick in are breakpoints another resource on fund costs is personalfundcom among other things the site helps tally the tax consequences of owning particular funds investors can look at a few funds free but a subscription to the site costs  a month or  a year in the face of industry competition and the rise of lowcost exchangetraded funds or etfs many funds are cutting the expenses that are automatically subtracted from fund assets that is good news for investors annua

#### 3. Parse article list into a white-space seperated word list retaining the article's word ordering. Exclude single-letter words.

Single letter words are excluded in our stopwords function

#### 4 Exclude articles with page citation tags corresponding to sections other than A, B, C or misssing

\Exclude articles corresponding to weekends



In [81]:
import sys
sys.path.append(r"/Users/sam/miniconda3/lib/python3.7/site-packages")
import dataset_utils
from xone import calendar

In [189]:
def business_dates(start, end):
    us_cal = calendar.USTradingCalendar()
    kw = dict(start=start, end=end)
    return pd.bdate_range(**kw).drop(us_cal.holidays(**kw))

business_dates = business_dates(start='2006-01-01', end='2017-12-31')

cleaning = merged[merged.datetime.isin(business_dates)]
cleaning

,link,text,year,month,day,title,author,datetime,alpha
23261,/docview/398949833/F358B7F8C6E64F12PQ/90?accou...,Consumers who want to earn US Airways frequen...,2006.0,1.0,3.0,Competing Card Aims to Land US Airways Fliers,"Lieber, Ron.",2006-01-03,consumers who want to earn us airways frequen...
23269,/docview/399004218/F358B7F8C6E64F12PQ/99?accou...,TOKYO -- If your New Year's resolutions inclu...,2006.0,1.0,3.0,Seaweed Booms for Dieters in Japan; 'Kanten' P...,"Morse, Andrew.",2006-01-03,tokyo if your new years resolutions include ...
23268,/docview/398949378/F358B7F8C6E64F12PQ/98?accou...,Reuters News Service A University of Californ...,2006.0,1.0,3.0,Obesity May Spur Renal Failure,NaN,2006-01-03,reuters news service a university of californ...
23267,/docview/399051935/F358B7F8C6E64F12PQ/97?accou...,[Columnist Tara Parker-Pope answers readers' ...,2006.0,1.0,3.0,Health Mailbox,"Parker-Pope, Tara.",2006-01-03,columnist tara parkerpope answers readers que...
23266,/docview/399000276/F358B7F8C6E64F12PQ/96?accou...,Associated Press A third of people suffering ...,2006.0,1.0,3.0,Antidepressant Use Is Evaluated,NaN,2006-01-03,associated press a third of people suffering ...
...,...,...,...,...,...,...,...,...,...
76138,/docview/1981625148/16FFCEEF54834D5CPQ/7?accou...,Islamic State said it was behind a suicide bo...,2017.0,12.0,29.0,World-Wide,NaN,2017-12-29,islamic state said it was behind a suicide bo...
102587,/docview/1981624558/16FFCEEF54834D5CPQ/70?acco...,Being a real-estate millionaire in London is ...,2017.0,12.0,29.0,MANSION --- London's Most Elite Streets --- On...,"Bloomfield, Ruth.",2017-12-29,being a realestate millionaire in london is s...
76137,/docview/1981624584/16FFCEEF54834D5CPQ/6?accou...,"Another Christmas has gone by and, once again...",2017.0,12.0,29.0,Who's Nuts for Fruitcake? Hikers and Other Ath...,"McKay, Betsy.",2017-12-29,another christmas has gone by and once again ...
76136,/docview/1981683991/16FFCEEF54834D5CPQ/5?accou...,SoftBank Group Corp. won its bid to buy a maj...,2017.0,12.0,29.0,SoftBank Grabs Uber Stake --- Group led by Jap...,"Bensinger, Greg",2017-12-29,softbank group corp won its bid to buy a majo...


\6 Exclude articles with subject tags associated with obviously non-economic content. 

Cannot access subject tags. However, we removed articles with these terms within the title in the datascraping process.



In [69]:
irrelevant = ['food & culture', 'football', "football's", "baseball", "baseball's", "nba", "mlb", "nfl", "mls",
              "nhl", "hockey", "hockey's", "yankees", "mets", "rangers", "soccer", "soccer's", "arts & entertainment",
              "arena", "jets'", "giants'", "sports", "arts & entertainment:", "heard & scene", "book", "books",
              "weekend journal", "off duty", "personal journal"]

\7. Exclude articles with certain headline patterns (such as associated with data tables or regular sports, leisure, or books columns)

Did our best to exclude in step 6


\8 Concatenate articles with the same accession number as these are chained articles. 

Don't have accession numbers

\9 Exclude articles iwth less than 100 words

In [190]:
print(len(cleaning))
cleaning['word_count'] = cleaning['text'].apply(lambda x: len(x.split()))
condition = cleaning['word_count'] > 99
temp = cleaning[condition]
cleaning = cleaning.reset_index(drop=True)
len(cleaning)

79910


79910

\10 remove "stop words" and URL based terms

Start by removing URL based terms

In [192]:
# using text column since that is the original article
cleaning['alpha'] = cleaning['alpha'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
cleaning

,link,text,year,month,day,title,author,datetime,alpha,word_count
0,/docview/398949833/F358B7F8C6E64F12PQ/90?accou...,Consumers who want to earn US Airways frequen...,2006.0,1.0,3.0,Competing Card Aims to Land US Airways Fliers,"Lieber, Ron.",2006-01-03,consumers who want to earn us airways frequen...,400
1,/docview/399004218/F358B7F8C6E64F12PQ/99?accou...,TOKYO -- If your New Year's resolutions inclu...,2006.0,1.0,3.0,Seaweed Booms for Dieters in Japan; 'Kanten' P...,"Morse, Andrew.",2006-01-03,tokyo if your new years resolutions include ...,782
2,/docview/398949378/F358B7F8C6E64F12PQ/98?accou...,Reuters News Service A University of Californ...,2006.0,1.0,3.0,Obesity May Spur Renal Failure,NaN,2006-01-03,reuters news service a university of californ...,229
3,/docview/399051935/F358B7F8C6E64F12PQ/97?accou...,[Columnist Tara Parker-Pope answers readers' ...,2006.0,1.0,3.0,Health Mailbox,"Parker-Pope, Tara.",2006-01-03,columnist tara parkerpope answers readers que...,784
4,/docview/399000276/F358B7F8C6E64F12PQ/96?accou...,Associated Press A third of people suffering ...,2006.0,1.0,3.0,Antidepressant Use Is Evaluated,NaN,2006-01-03,associated press a third of people suffering ...,718
...,...,...,...,...,...,...,...,...,...,...
79905,/docview/1981625148/16FFCEEF54834D5CPQ/7?accou...,Islamic State said it was behind a suicide bo...,2017.0,12.0,29.0,World-Wide,NaN,2017-12-29,islamic state said it was behind a suicide bo...,195
79906,/docview/1981624558/16FFCEEF54834D5CPQ/70?acco...,Being a real-estate millionaire in London is ...,2017.0,12.0,29.0,MANSION --- London's Most Elite Streets --- On...,"Bloomfield, Ruth.",2017-12-29,being a realestate millionaire in london is s...,1068
79907,/docview/1981624584/16FFCEEF54834D5CPQ/6?accou...,"Another Christmas has gone by and, once again...",2017.0,12.0,29.0,Who's Nuts for Fruitcake? Hikers and Other Ath...,"McKay, Betsy.",2017-12-29,another christmas has gone by and once again ...,908
79908,/docview/1981683991/16FFCEEF54834D5CPQ/5?accou...,SoftBank Group Corp. won its bid to buy a maj...,2017.0,12.0,29.0,SoftBank Grabs Uber Stake --- Group led by Jap...,"Bensinger, Greg",2017-12-29,softbank group corp won its bid to buy a majo...,1104


Remove stopwords (not yet)

In [131]:
# def remove_stopwords(corpus):
#     print("getting ready")
#     en = spacy.load('en_core_web_sm')
#     stopwords = en.Defaults.stop_words
#     texts_out = []
#     article = [line for line in corpus]
#     split_articles = [sentence.split(" ") for sentence in article]
#     count = 0
#     for item in split_articles:
#         tokens_filtered = [token for token in item if token not in stopwords]
#         final = (" ").join(tokens_filtered)
#         texts_out.append(final)
#         count += 1
#         if count % 100 == 0:
#             print(count)
#     return texts_out

In [135]:
#no_stops = remove_stopwords(cleaning['text'])


# no_stops = remove_stopwords(cleaning['text'])
# no_stops
# test['text'] = cleaning['text'].apply(lambda x: remove_stopwords(x))
# test

getting ready
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18

['Consumers want earn US Airways frequent-flier miles credit cards competing companies pitching plastic. Today, Barclays PLC\'s U.S. credit-card unit launch US Airways Dividend Miles World MasterCard. Meanwhile, hundreds thousands people Bank America Corp.\'s U.S. Airways Visa Signature Card. This unusual state affairs world airline-card partnerships card issuers usually lock airlines exclusive contracts. And it\'s potentially confusing consumers, don\'t know longer Bank America cards work advantage bonuses Barclays dangling switch. When America West Holdings Corp. US Airways Group Inc. merged year, airline credit card. The combined company cut single, new card deal. Barclays won bidding issue card marketing it, Bank America allowed continue issue US Airways cards Dec. 27, 2007, according US Airways marketing executive Scott Kirby. After that, it\'s unclear happen existing Bank America cards, Mr. Kirby says. A Bank America spokeswoman declined comment counter-offers retain customers sa

In [141]:
# test['no_stops'] = no_stops
# for i in test['no_stops'].iloc[0].split():
#     print(i)

Consumers
want
earn
US
Airways
frequent-flier
miles
credit
cards
competing
companies
pitching
plastic.
Today,
Barclays
PLC's
U.S.
credit-card
unit
launch
US
Airways
Dividend
Miles
World
MasterCard.
Meanwhile,
hundreds
thousands
people
Bank
America
Corp.'s
U.S.
Airways
Visa
Signature
Card.
This
unusual
state
affairs
world
airline-card
partnerships
card
issuers
usually
lock
airlines
exclusive
contracts.
And
it's
potentially
confusing
consumers,
don't
know
longer
Bank
America
cards
work
advantage
bonuses
Barclays
dangling
switch.
When
America
West
Holdings
Corp.
US
Airways
Group
Inc.
merged
year,
airline
credit
card.
The
combined
company
cut
single,
new
card
deal.
Barclays
won
bidding
issue
card
marketing
it,
Bank
America
allowed
continue
issue
US
Airways
cards
Dec.
27,
2007,
according
US
Airways
marketing
executive
Scott
Kirby.
After
that,
it's
unclear
happen
existing
Bank
America
cards,
Mr.
Kirby
says.
A
Bank
America
spokeswoman
declined
comment
counter-offers
retain
customers
said
spec

\11. Lemmatization

We are doing lemmatization first because some stopwords are not removed before this step due to SpaCy processes.

They have a specific list of rules in which they lemmatized. They do not appear to differ significantly from the rules that spacy applies. Due to the computational complexity of lemmatizing this corpus, we will apply Spacy's lemmatizer due to its computational speed and accuracy. 

In [193]:
def lemmatization(texts): # to keep with paper we will not remove any post tags
    # import the English spacy object as en
    en = spacy.load('en_core_web_sm', disable=["parser", "ner"])
    texts_out = []
    lemma_text = []
    # Lemmatize text
    count = 0 # for keeping track
    for article in texts:
        tokens = en(article)#[en(text) for text in article]
        [lemma_text.append(tok.lemma_) for tok in tokens] # lemmatize token if in allowed_postags
        final = " ".join(lemma_text)
        texts_out.append(final)
        lemma_text = []
        count+=1
        if count % 100 == 0:
            print(count)
    return texts_out



In [203]:
lemmatized = lemmatization(cleaning['alpha'])

# en = spacy.load('en_core_web_sm', disable=["parser", "ner"])
# lemma = en("lemmatizer")
# en.get_pipe('lemmatizer').lookups.get_table("lemma_exc")#["noun"]["data"]

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [206]:
lemmatized_v2 = lemmatized

In [207]:
import pickle
with open('WSJ_lemmatized_v2.pkl', 'wb') as f:
    pickle.dump(lemmatized_v2, f)

In [208]:
lemmatized_v2[4]

'  associate press a third of people suffer serious depression be help with the first antidepressant but closely tailor the treatment appear to be a crucial key to success accord to initial result of an eagerly await we study on the drug patient whose depression symptom disappear take high than typical drug dose but perhaps most important they receive close monitoring and frequent dose adjustment in the first three month   a level of care that few us patient today receive the main goal of the governmentfunded study be to identify what hardertotreat patient should try when initial treatment fail instead of abandon therapy in frustration those result be due in a few month psychiatrist have long know that for most depression sufferer the first antidepressant choice will not be a panacea just as patient with epilepsy heart disease or cancer often must mix and match medication before find the good choice physician have have little scientific evidence to guide their choice of antidepressant 

Now we will remove the stopwords from the lemmatized texts

In [209]:
def remove_stopwords(corpus):
    print("getting ready")
    en = spacy.load('en_core_web_sm')
    stopwords = en.Defaults.stop_words
    texts_out = []
    article = [line for line in corpus]
    split_articles = [sentence.split(" ") for sentence in article]
    count = 0
    for item in split_articles:
        tokens_filtered = [token for token in item if token not in stopwords]
        final = (" ").join(tokens_filtered)
        texts_out.append(final)
        count += 1
        if count % 100 == 0:
            print(count)
    return texts_out

In [210]:
cleaned_text = remove_stopwords(lemmatized_v2)
cleaned_text[0][0:20]

getting ready
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18

'  consumer want earn'

In [422]:
cleaning = pd.read_pickle('cleaned_WSJ_data.pkl') 
cleaning

,link,text,year,month,day,title,author,datetime,alpha,word_count,lemmatized_and_stops,data_bigrams,corpus
0,/docview/398949833/F358B7F8C6E64F12PQ/90?accou...,Consumers who want to earn US Airways frequen...,2006.0,1.0,3.0,Competing Card Aims to Land US Airways Fliers,"Lieber, Ron.",2006-01-03,consumers who want to earn us airways frequen...,400,consumer want earn airways frequentflier mil...,"[consumer, want, earn, airways, frequentflier,...","[(0, 1), (1, 1), (2, 1), (3, 2), (4, 2), (5, 8..."
1,/docview/399004218/F358B7F8C6E64F12PQ/99?accou...,TOKYO -- If your New Year's resolutions inclu...,2006.0,1.0,3.0,Seaweed Booms for Dieters in Japan; 'Kanten' P...,"Morse, Andrew.",2006-01-03,tokyo if your new years resolutions include ...,782,tokyo new year resolution include slimme w...,"[tokyo, new, year, resolution, include, slimme...","[(0, 1), (6, 1), (12, 2), (21, 1), (24, 2), (2..."
2,/docview/398949378/F358B7F8C6E64F12PQ/98?accou...,Reuters News Service A University of Californ...,2006.0,1.0,3.0,Obesity May Spur Renal Failure,NaN,2006-01-03,reuters news service a university of californ...,229,reuters news service university california s...,"[reuters, news, service, university, californi...","[(60, 1), (63, 1), (80, 1), (86, 3), (98, 1), ..."
3,/docview/399051935/F358B7F8C6E64F12PQ/97?accou...,[Columnist Tara Parker-Pope answers readers' ...,2006.0,1.0,3.0,Health Mailbox,"Parker-Pope, Tara.",2006-01-03,columnist tara parkerpope answers readers que...,784,columnist tara parkerpope answer reader ques...,"[columnist_tara, parkerpope_answer, reader, qu...","[(17, 1), (19, 1), (24, 1), (38, 1), (54, 1), ..."
4,/docview/399000276/F358B7F8C6E64F12PQ/96?accou...,Associated Press A third of people suffering ...,2006.0,1.0,3.0,Antidepressant Use Is Evaluated,NaN,2006-01-03,associated press a third of people suffering ...,718,associate press people suffer depression hel...,"[associate_press, people, suffer, depression, ...","[(0, 1), (6, 1), (54, 1), (58, 1), (63, 3), (6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79905,/docview/1981625148/16FFCEEF54834D5CPQ/7?accou...,Islamic State said it was behind a suicide bo...,2017.0,12.0,29.0,World-Wide,NaN,2017-12-29,islamic state said it was behind a suicide bo...,195,islamic state suicide bombing shiite muslim ...,"[islamic, state, suicide_bombing, shiite, musl...","[(40, 1), (79, 2), (86, 1), (94, 1), (103, 1),..."
79906,/docview/1981624558/16FFCEEF54834D5CPQ/70?acco...,Being a real-estate millionaire in London is ...,2017.0,12.0,29.0,MANSION --- London's Most Elite Streets --- On...,"Bloomfield, Ruth.",2017-12-29,being a realestate millionaire in london is s...,1068,realestate millionaire london passe london ...,"[realestate, millionaire, london, passe, londo...","[(7, 1), (21, 2), (30, 2), (33, 3), (37, 1), (..."
79907,/docview/1981624584/16FFCEEF54834D5CPQ/6?accou...,"Another Christmas has gone by and, once again...",2017.0,12.0,29.0,Who's Nuts for Fruitcake? Hikers and Other Ath...,"McKay, Betsy.",2017-12-29,another christmas has gone by and once again ...,908,christmas uneaten fruitcake s welcome news j...,"[christmas, uneaten, fruitcake, welcome, news,...","[(17, 1), (21, 1), (30, 1), (47, 1), (58, 1), ..."
79908,/docview/1981683991/16FFCEEF54834D5CPQ/5?accou...,SoftBank Group Corp. won its bid to buy a maj...,2017.0,12.0,29.0,SoftBank Grabs Uber Stake --- Group led by Jap...,"Bensinger, Greg",2017-12-29,softbank group corp won its bid to buy a majo...,1104,softbank group corp win bid buy major stake ...,"[softbank, group, corp, win, bid, buy, major, ...","[(6, 1), (11, 1), (12, 3), (17, 3), (19, 1), (..."


In [423]:
cleaning['lemmatized_and_stops'] = cleaned_text
cleaning

,link,text,year,month,day,title,author,datetime,alpha,word_count,lemmatized_and_stops,data_bigrams,corpus
0,/docview/398949833/F358B7F8C6E64F12PQ/90?accou...,Consumers who want to earn US Airways frequen...,2006.0,1.0,3.0,Competing Card Aims to Land US Airways Fliers,"Lieber, Ron.",2006-01-03,consumers who want to earn us airways frequen...,400,consumer want earn airways frequentflier mil...,"[consumer, want, earn, airways, frequentflier,...","[(0, 1), (1, 1), (2, 1), (3, 2), (4, 2), (5, 8..."
1,/docview/399004218/F358B7F8C6E64F12PQ/99?accou...,TOKYO -- If your New Year's resolutions inclu...,2006.0,1.0,3.0,Seaweed Booms for Dieters in Japan; 'Kanten' P...,"Morse, Andrew.",2006-01-03,tokyo if your new years resolutions include ...,782,tokyo new year resolution include slimme w...,"[tokyo, new, year, resolution, include, slimme...","[(0, 1), (6, 1), (12, 2), (21, 1), (24, 2), (2..."
2,/docview/398949378/F358B7F8C6E64F12PQ/98?accou...,Reuters News Service A University of Californ...,2006.0,1.0,3.0,Obesity May Spur Renal Failure,NaN,2006-01-03,reuters news service a university of californ...,229,reuters news service university california s...,"[reuters, news, service, university, californi...","[(60, 1), (63, 1), (80, 1), (86, 3), (98, 1), ..."
3,/docview/399051935/F358B7F8C6E64F12PQ/97?accou...,[Columnist Tara Parker-Pope answers readers' ...,2006.0,1.0,3.0,Health Mailbox,"Parker-Pope, Tara.",2006-01-03,columnist tara parkerpope answers readers que...,784,columnist tara parkerpope answer reader ques...,"[columnist_tara, parkerpope_answer, reader, qu...","[(17, 1), (19, 1), (24, 1), (38, 1), (54, 1), ..."
4,/docview/399000276/F358B7F8C6E64F12PQ/96?accou...,Associated Press A third of people suffering ...,2006.0,1.0,3.0,Antidepressant Use Is Evaluated,NaN,2006-01-03,associated press a third of people suffering ...,718,associate press people suffer depression hel...,"[associate_press, people, suffer, depression, ...","[(0, 1), (6, 1), (54, 1), (58, 1), (63, 3), (6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79905,/docview/1981625148/16FFCEEF54834D5CPQ/7?accou...,Islamic State said it was behind a suicide bo...,2017.0,12.0,29.0,World-Wide,NaN,2017-12-29,islamic state said it was behind a suicide bo...,195,islamic state suicide bombing shiite muslim ...,"[islamic, state, suicide_bombing, shiite, musl...","[(40, 1), (79, 2), (86, 1), (94, 1), (103, 1),..."
79906,/docview/1981624558/16FFCEEF54834D5CPQ/70?acco...,Being a real-estate millionaire in London is ...,2017.0,12.0,29.0,MANSION --- London's Most Elite Streets --- On...,"Bloomfield, Ruth.",2017-12-29,being a realestate millionaire in london is s...,1068,realestate millionaire london passe london ...,"[realestate, millionaire, london, passe, londo...","[(7, 1), (21, 2), (30, 2), (33, 3), (37, 1), (..."
79907,/docview/1981624584/16FFCEEF54834D5CPQ/6?accou...,"Another Christmas has gone by and, once again...",2017.0,12.0,29.0,Who's Nuts for Fruitcake? Hikers and Other Ath...,"McKay, Betsy.",2017-12-29,another christmas has gone by and once again ...,908,christmas uneaten fruitcake s welcome news j...,"[christmas, uneaten, fruitcake, welcome, news,...","[(17, 1), (21, 1), (30, 1), (47, 1), (58, 1), ..."
79908,/docview/1981683991/16FFCEEF54834D5CPQ/5?accou...,SoftBank Group Corp. won its bid to buy a maj...,2017.0,12.0,29.0,SoftBank Grabs Uber Stake --- Group led by Jap...,"Bensinger, Greg",2017-12-29,softbank group corp won its bid to buy a majo...,1104,softbank group corp win bid buy major stake ...,"[softbank, group, corp, win, bid, buy, major, ...","[(6, 1), (11, 1), (12, 3), (17, 3), (19, 1), (..."


Remove words with less than 3 characters as per the lemmatization step

In [424]:
cleaning['lemmatized_and_stops'] = cleaning['lemmatized_and_stops'].str.replace(r'\b\w{1,2}\b', '', regex=True)


In [427]:
cleaning

,link,text,year,month,day,title,author,datetime,alpha,word_count,lemmatized_and_stops,data_bigrams,corpus
0,/docview/398949833/F358B7F8C6E64F12PQ/90?accou...,Consumers who want to earn US Airways frequen...,2006.0,1.0,3.0,Competing Card Aims to Land US Airways Fliers,"Lieber, Ron.",2006-01-03,consumers who want to earn us airways frequen...,400,consumer want earn airways frequentflier mil...,"[consumer, want, earn, airways, frequentflier,...","[(0, 1), (1, 1), (2, 1), (3, 2), (4, 2), (5, 8..."
1,/docview/399004218/F358B7F8C6E64F12PQ/99?accou...,TOKYO -- If your New Year's resolutions inclu...,2006.0,1.0,3.0,Seaweed Booms for Dieters in Japan; 'Kanten' P...,"Morse, Andrew.",2006-01-03,tokyo if your new years resolutions include ...,782,tokyo new year resolution include slimme w...,"[tokyo, new, year, resolution, include, slimme...","[(0, 1), (6, 1), (12, 2), (21, 1), (24, 2), (2..."
2,/docview/398949378/F358B7F8C6E64F12PQ/98?accou...,Reuters News Service A University of Californ...,2006.0,1.0,3.0,Obesity May Spur Renal Failure,NaN,2006-01-03,reuters news service a university of californ...,229,reuters news service university california s...,"[reuters, news, service, university, californi...","[(60, 1), (63, 1), (80, 1), (86, 3), (98, 1), ..."
3,/docview/399051935/F358B7F8C6E64F12PQ/97?accou...,[Columnist Tara Parker-Pope answers readers' ...,2006.0,1.0,3.0,Health Mailbox,"Parker-Pope, Tara.",2006-01-03,columnist tara parkerpope answers readers que...,784,columnist tara parkerpope answer reader ques...,"[columnist_tara, parkerpope_answer, reader, qu...","[(17, 1), (19, 1), (24, 1), (38, 1), (54, 1), ..."
4,/docview/399000276/F358B7F8C6E64F12PQ/96?accou...,Associated Press A third of people suffering ...,2006.0,1.0,3.0,Antidepressant Use Is Evaluated,NaN,2006-01-03,associated press a third of people suffering ...,718,associate press people suffer depression hel...,"[associate_press, people, suffer, depression, ...","[(0, 1), (6, 1), (54, 1), (58, 1), (63, 3), (6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79905,/docview/1981625148/16FFCEEF54834D5CPQ/7?accou...,Islamic State said it was behind a suicide bo...,2017.0,12.0,29.0,World-Wide,NaN,2017-12-29,islamic state said it was behind a suicide bo...,195,islamic state suicide bombing shiite muslim ...,"[islamic, state, suicide_bombing, shiite, musl...","[(40, 1), (79, 2), (86, 1), (94, 1), (103, 1),..."
79906,/docview/1981624558/16FFCEEF54834D5CPQ/70?acco...,Being a real-estate millionaire in London is ...,2017.0,12.0,29.0,MANSION --- London's Most Elite Streets --- On...,"Bloomfield, Ruth.",2017-12-29,being a realestate millionaire in london is s...,1068,realestate millionaire london passe london ...,"[realestate, millionaire, london, passe, londo...","[(7, 1), (21, 2), (30, 2), (33, 3), (37, 1), (..."
79907,/docview/1981624584/16FFCEEF54834D5CPQ/6?accou...,"Another Christmas has gone by and, once again...",2017.0,12.0,29.0,Who's Nuts for Fruitcake? Hikers and Other Ath...,"McKay, Betsy.",2017-12-29,another christmas has gone by and once again ...,908,christmas uneaten fruitcake welcome news ja...,"[christmas, uneaten, fruitcake, welcome, news,...","[(17, 1), (21, 1), (30, 1), (47, 1), (58, 1), ..."
79908,/docview/1981683991/16FFCEEF54834D5CPQ/5?accou...,SoftBank Group Corp. won its bid to buy a maj...,2017.0,12.0,29.0,SoftBank Grabs Uber Stake --- Group led by Jap...,"Bensinger, Greg",2017-12-29,softbank group corp won its bid to buy a majo...,1104,softbank group corp win bid buy major stake ...,"[softbank, group, corp, win, bid, buy, major, ...","[(6, 1), (11, 1), (12, 3), (17, 3), (19, 1), (..."


We will now do a quick gensim.preprocess to clean the data. This is part of step 14 (Vectorizing). To use Gensim to create bigrams, this step should happen here.

In [428]:
def gen_words(texts): # pass in your texts as an argument
    final = []
    count = 0
    for text in texts: # iterate over the texts
        new = gensim.utils.simple_preprocess(text, deacc=True) 
        final.append(new)
        count+=1
        if count % 1000 == 0:
            print(count)
    return final

In [429]:
cleaned_text = cleaning['lemmatized_and_stops']
preprocessed = gen_words(cleaned_text)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000


In [430]:
preprocessed[0]

['consumer',
 'want',
 'earn',
 'airways',
 'frequentflier',
 'mile',
 'credit',
 'card',
 'compete',
 'company',
 'pitch',
 'plastic',
 'today',
 'barclay',
 'plc',
 'creditcard',
 'unit',
 'launch',
 'airways',
 'dividend',
 'miles',
 'world',
 'mastercard',
 'thousand',
 'people',
 'use',
 'bank',
 'america',
 'corps',
 'airways',
 'visa',
 'signature',
 'card',
 'unusual',
 'state',
 'affair',
 'world',
 'airlinecard',
 'partnership',
 'card',
 'issuer',
 'usually',
 'lock',
 'airline',
 'exclusive',
 'contract',
 'potentially',
 'confusing',
 'consumer',
 'know',
 'long',
 'bank',
 'america',
 'card',
 'work',
 'advantage',
 'bonus',
 'barclay',
 'dangle',
 'switch',
 'america',
 'west',
 'holdings',
 'corp',
 'airways',
 'group',
 'inc',
 'merge',
 'year',
 'airline',
 'credit',
 'card',
 'combine',
 'company',
 'cut',
 'single',
 'new',
 'card',
 'deal',
 'barclay',
 'win',
 'bidding',
 'issue',
 'card',
 'market',
 'bank',
 'america',
 'allow',
 'continue',
 'issue',
 'airways'

\12. From the resulting uni-grams, generate the set of bigrams. (Note that hyphenated words become unigrams from step 1, which removes all punctuation.

The authors do not specify the threshold that they use. We will use a threshold of 200 since this is a large corpus. This means that the words must appear together 200 times to be considered a bigram

In [431]:
bigram_phrases = gensim.models.Phrases(preprocessed,
                                       min_count=5,
                                       threshold=200) # play around with the threshold

# create our bigrams and trigrams
bigram = gensim.models.phrases.Phraser(bigram_phrases)

# need a function to reinject these into your corpus
def make_bigrams(texts):
    return [bigram[doc] for doc in texts] # identify and reconsitute all of your bigrams

# call your function to make a data object that includes your bigrams
data_bigrams = make_bigrams(preprocessed)
data_bigrams[0:3]

[['consumer',
  'want',
  'earn',
  'airways',
  'frequentflier',
  'mile',
  'credit',
  'card',
  'compete',
  'company',
  'pitch',
  'plastic',
  'today',
  'barclay',
  'plc',
  'creditcard',
  'unit',
  'launch',
  'airways',
  'dividend',
  'miles',
  'world',
  'mastercard',
  'thousand',
  'people',
  'use',
  'bank',
  'america',
  'corps',
  'airways',
  'visa',
  'signature',
  'card',
  'unusual',
  'state',
  'affair',
  'world',
  'airlinecard',
  'partnership',
  'card',
  'issuer',
  'usually',
  'lock',
  'airline',
  'exclusive',
  'contract',
  'potentially',
  'confusing',
  'consumer',
  'know',
  'long',
  'bank',
  'america',
  'card',
  'work',
  'advantage',
  'bonus',
  'barclay',
  'dangle',
  'switch',
  'america',
  'west',
  'holdings',
  'corp',
  'airways',
  'group',
  'inc',
  'merge',
  'year',
  'airline',
  'credit',
  'card',
  'combine',
  'company',
  'cut',
  'single',
  'new',
  'card',
  'deal',
  'barclay',
  'win',
  'bidding',
  'issue',
 

In [432]:
# append our data_bigrams to our dataframe
cleaning['data_bigrams'] = data_bigrams
cleaning

,link,text,year,month,day,title,author,datetime,alpha,word_count,lemmatized_and_stops,data_bigrams,corpus
0,/docview/398949833/F358B7F8C6E64F12PQ/90?accou...,Consumers who want to earn US Airways frequen...,2006.0,1.0,3.0,Competing Card Aims to Land US Airways Fliers,"Lieber, Ron.",2006-01-03,consumers who want to earn us airways frequen...,400,consumer want earn airways frequentflier mil...,"[consumer, want, earn, airways, frequentflier,...","[(0, 1), (1, 1), (2, 1), (3, 2), (4, 2), (5, 8..."
1,/docview/399004218/F358B7F8C6E64F12PQ/99?accou...,TOKYO -- If your New Year's resolutions inclu...,2006.0,1.0,3.0,Seaweed Booms for Dieters in Japan; 'Kanten' P...,"Morse, Andrew.",2006-01-03,tokyo if your new years resolutions include ...,782,tokyo new year resolution include slimme w...,"[tokyo, new, year, resolution, include, slimme...","[(0, 1), (6, 1), (12, 2), (21, 1), (24, 2), (2..."
2,/docview/398949378/F358B7F8C6E64F12PQ/98?accou...,Reuters News Service A University of Californ...,2006.0,1.0,3.0,Obesity May Spur Renal Failure,NaN,2006-01-03,reuters news service a university of californ...,229,reuters news service university california s...,"[reuters, news, service, university, californi...","[(60, 1), (63, 1), (80, 1), (86, 3), (98, 1), ..."
3,/docview/399051935/F358B7F8C6E64F12PQ/97?accou...,[Columnist Tara Parker-Pope answers readers' ...,2006.0,1.0,3.0,Health Mailbox,"Parker-Pope, Tara.",2006-01-03,columnist tara parkerpope answers readers que...,784,columnist tara parkerpope answer reader ques...,"[columnist_tara, parkerpope_answer, reader, qu...","[(17, 1), (19, 1), (24, 1), (38, 1), (54, 1), ..."
4,/docview/399000276/F358B7F8C6E64F12PQ/96?accou...,Associated Press A third of people suffering ...,2006.0,1.0,3.0,Antidepressant Use Is Evaluated,NaN,2006-01-03,associated press a third of people suffering ...,718,associate press people suffer depression hel...,"[associate_press, people, suffer, depression, ...","[(0, 1), (6, 1), (54, 1), (58, 1), (63, 3), (6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79905,/docview/1981625148/16FFCEEF54834D5CPQ/7?accou...,Islamic State said it was behind a suicide bo...,2017.0,12.0,29.0,World-Wide,NaN,2017-12-29,islamic state said it was behind a suicide bo...,195,islamic state suicide bombing shiite muslim ...,"[islamic, state, suicide_bombing, shiite, musl...","[(40, 1), (79, 2), (86, 1), (94, 1), (103, 1),..."
79906,/docview/1981624558/16FFCEEF54834D5CPQ/70?acco...,Being a real-estate millionaire in London is ...,2017.0,12.0,29.0,MANSION --- London's Most Elite Streets --- On...,"Bloomfield, Ruth.",2017-12-29,being a realestate millionaire in london is s...,1068,realestate millionaire london passe london ...,"[realestate, millionaire, london, passe, londo...","[(7, 1), (21, 2), (30, 2), (33, 3), (37, 1), (..."
79907,/docview/1981624584/16FFCEEF54834D5CPQ/6?accou...,"Another Christmas has gone by and, once again...",2017.0,12.0,29.0,Who's Nuts for Fruitcake? Hikers and Other Ath...,"McKay, Betsy.",2017-12-29,another christmas has gone by and once again ...,908,christmas uneaten fruitcake welcome news ja...,"[christmas, uneaten, fruitcake, welcome, news,...","[(17, 1), (21, 1), (30, 1), (47, 1), (58, 1), ..."
79908,/docview/1981683991/16FFCEEF54834D5CPQ/5?accou...,SoftBank Group Corp. won its bid to buy a maj...,2017.0,12.0,29.0,SoftBank Grabs Uber Stake --- Group led by Jap...,"Bensinger, Greg",2017-12-29,softbank group corp won its bid to buy a majo...,1104,softbank group corp win bid buy major stake ...,"[softbank, group, corp, win, bid, buy, major, ...","[(6, 1), (11, 1), (12, 3), (17, 3), (19, 1), (..."


13. Exclude terms that appear in less than 0.1% of articles

To do this, we have to first create our BOW model

In [433]:
from gensim.models import TfidfModel

# make a dictionary of how many times a word occurs in the corpus
id2word = corpora.Dictionary(data_bigrams)
id2word = id2word # fulfills step 13, which excludes terms that occur in less that 0.1% of articles
print("There are {num} words in the corpus before pruning".format(num = (len(id2word))))
id2word.filter_extremes(no_above=8)
print("There are {num} words in the corpus after pruning".format(num = (len(id2word))))
# rename data_bigrams_trigrams to something easier to say (not necessary)
texts = data_bigrams



There are 266423 words in the corpus before pruning
There are 76056 words in the corpus after pruning


14. Convert an article's word list into a vector of counts for each term in the vocabulary. This is the final step. 

In [434]:
# this converts your texts into a bag of words model
corpus = [id2word.doc2bow(text) for text in texts] 
print(corpus[1][0:20]) # We have created a tuple (index of word, # of times it appears in the corpus)

[(0, 1), (6, 1), (12, 2), (21, 1), (24, 2), (27, 1), (54, 5), (58, 1), (63, 2), (78, 2), (82, 2), (85, 1), (91, 3), (113, 4), (117, 1), (122, 5), (123, 1), (124, 1), (125, 1), (126, 1)]


In [435]:
# append to our final dataframe

cleaned_data = cleaning
cleaned_data['corpus'] = corpus
cleaned_data

,link,text,year,month,day,title,author,datetime,alpha,word_count,lemmatized_and_stops,data_bigrams,corpus
0,/docview/398949833/F358B7F8C6E64F12PQ/90?accou...,Consumers who want to earn US Airways frequen...,2006.0,1.0,3.0,Competing Card Aims to Land US Airways Fliers,"Lieber, Ron.",2006-01-03,consumers who want to earn us airways frequen...,400,consumer want earn airways frequentflier mil...,"[consumer, want, earn, airways, frequentflier,...","[(0, 1), (1, 1), (2, 1), (3, 2), (4, 2), (5, 8..."
1,/docview/399004218/F358B7F8C6E64F12PQ/99?accou...,TOKYO -- If your New Year's resolutions inclu...,2006.0,1.0,3.0,Seaweed Booms for Dieters in Japan; 'Kanten' P...,"Morse, Andrew.",2006-01-03,tokyo if your new years resolutions include ...,782,tokyo new year resolution include slimme w...,"[tokyo, new, year, resolution, include, slimme...","[(0, 1), (6, 1), (12, 2), (21, 1), (24, 2), (2..."
2,/docview/398949378/F358B7F8C6E64F12PQ/98?accou...,Reuters News Service A University of Californ...,2006.0,1.0,3.0,Obesity May Spur Renal Failure,NaN,2006-01-03,reuters news service a university of californ...,229,reuters news service university california s...,"[reuters, news, service, university, californi...","[(60, 1), (63, 1), (79, 1), (85, 3), (97, 1), ..."
3,/docview/399051935/F358B7F8C6E64F12PQ/97?accou...,[Columnist Tara Parker-Pope answers readers' ...,2006.0,1.0,3.0,Health Mailbox,"Parker-Pope, Tara.",2006-01-03,columnist tara parkerpope answers readers que...,784,columnist tara parkerpope answer reader ques...,"[columnist_tara, parkerpope_answer, reader, qu...","[(17, 1), (19, 1), (24, 1), (38, 1), (54, 1), ..."
4,/docview/399000276/F358B7F8C6E64F12PQ/96?accou...,Associated Press A third of people suffering ...,2006.0,1.0,3.0,Antidepressant Use Is Evaluated,NaN,2006-01-03,associated press a third of people suffering ...,718,associate press people suffer depression hel...,"[associate_press, people, suffer, depression, ...","[(0, 1), (6, 1), (54, 1), (58, 1), (63, 3), (6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79905,/docview/1981625148/16FFCEEF54834D5CPQ/7?accou...,Islamic State said it was behind a suicide bo...,2017.0,12.0,29.0,World-Wide,NaN,2017-12-29,islamic state said it was behind a suicide bo...,195,islamic state suicide bombing shiite muslim ...,"[islamic, state, suicide_bombing, shiite, musl...","[(40, 1), (78, 2), (85, 1), (93, 1), (102, 1),..."
79906,/docview/1981624558/16FFCEEF54834D5CPQ/70?acco...,Being a real-estate millionaire in London is ...,2017.0,12.0,29.0,MANSION --- London's Most Elite Streets --- On...,"Bloomfield, Ruth.",2017-12-29,being a realestate millionaire in london is s...,1068,realestate millionaire london passe london ...,"[realestate, millionaire, london, passe, londo...","[(7, 1), (21, 2), (30, 2), (33, 3), (37, 1), (..."
79907,/docview/1981624584/16FFCEEF54834D5CPQ/6?accou...,"Another Christmas has gone by and, once again...",2017.0,12.0,29.0,Who's Nuts for Fruitcake? Hikers and Other Ath...,"McKay, Betsy.",2017-12-29,another christmas has gone by and once again ...,908,christmas uneaten fruitcake welcome news ja...,"[christmas, uneaten, fruitcake, welcome, news,...","[(17, 1), (21, 1), (30, 1), (47, 1), (58, 1), ..."
79908,/docview/1981683991/16FFCEEF54834D5CPQ/5?accou...,SoftBank Group Corp. won its bid to buy a maj...,2017.0,12.0,29.0,SoftBank Grabs Uber Stake --- Group led by Jap...,"Bensinger, Greg",2017-12-29,softbank group corp won its bid to buy a majo...,1104,softbank group corp win bid buy major stake ...,"[softbank, group, corp, win, bid, buy, major, ...","[(6, 1), (11, 1), (12, 3), (17, 3), (19, 1), (..."


In [436]:
# save to be sent to Matt

# save id2word
import pickle
with open('/Users/sam/PycharmProjects/Eco2460/WSJ_id2word.pkl', 'wb') as f:
    pickle.dump(id2word, f)

In [437]:
cleaned_data.to_pickle('/Users/sam/PycharmProjects/Eco2460/cleaned_WSJ_data.pkl')